In [1]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from sklearn.model_selection import train_test_split

### Load the dataset and pre-process them into tensors

In [2]:
data_set = pd.read_csv("data/cipher_objective.csv")
df_x = data_set["text"]
df_y = data_set["class"]

arr_x = []
arr_y = []

for i in range(len(df_x)):
    text = df_x.iloc[i]
    temp = [ord(c) for c in text]
    arr_x.append(temp)

    label = df_y.iloc[i]
    arr_y.append(0 if label == "exit" else 1)

data_x = torch.tensor(arr_x, dtype=torch.float32)
data_y = torch.tensor(arr_y, dtype=torch.long)

### Split the dataset and load them into data loaders

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
  data_x, data_y,
  test_size=0.4,
  stratify=data_y,
  random_state=42
)

train_x = X_train.detach().clone()
train_y = y_train.detach().clone()
test_x = X_test.detach().clone()
test_y = y_test.detach().clone()

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

torch.Size([2400, 100]) torch.Size([2400]) torch.Size([1600, 100]) torch.Size([1600])


In [4]:
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1600, shuffle=True)

### Training part

In [5]:
class Model(nn.Module):
  def __init__(self, input_size: int):
    super().__init__()
    self.l1 = nn.Linear(input_size, 1, True)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x: torch.Tensor):
    x = self.l1(x)
    return x

In [6]:
def train_model(loader: torch.utils.data.DataLoader, model: nn.Module):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model.to(device)
    optimiser = torch.optim.SGD(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()  

    epoch_losses = []
    for i in range(300):
        epoch_loss = 0
        model.train()
        for _, (x, y) in enumerate(loader):
            # x, y = x.to(device), y.to(device)

            optimiser.zero_grad()
            y_pred = model(x)
            y = y.view(-1, 1).long()   
            loss = loss_fn(y_pred, y.float())
            epoch_loss += loss.item()
            loss.backward()
            optimiser.step()

        epoch_loss = epoch_loss / len(loader)
        epoch_losses.append(epoch_loss)
        print("Epoch: {}, Loss: {}".format(i, epoch_loss))
        

    return model, epoch_losses

In [7]:
model = Model(100)
train_model(train_loader, model)

Epoch: 0, Loss: 74.3392988204956
Epoch: 1, Loss: 83.12007522583008
Epoch: 2, Loss: 80.28693923950195
Epoch: 3, Loss: 75.65507049560547
Epoch: 4, Loss: 76.28227291107177
Epoch: 5, Loss: 72.65942840576172
Epoch: 6, Loss: 71.7201473236084
Epoch: 7, Loss: 68.56741142272949
Epoch: 8, Loss: 67.76617736816407
Epoch: 9, Loss: 64.4251594543457
Epoch: 10, Loss: 69.57161254882813
Epoch: 11, Loss: 61.711208724975585
Epoch: 12, Loss: 63.781436157226565
Epoch: 13, Loss: 62.946955871582034
Epoch: 14, Loss: 61.13170280456543
Epoch: 15, Loss: 55.6065975189209
Epoch: 16, Loss: 55.38992862701416
Epoch: 17, Loss: 55.69268112182617
Epoch: 18, Loss: 52.65674819946289
Epoch: 19, Loss: 57.7535587310791
Epoch: 20, Loss: 51.32097473144531
Epoch: 21, Loss: 53.903232383728025
Epoch: 22, Loss: 47.677008628845215
Epoch: 23, Loss: 48.27898254394531
Epoch: 24, Loss: 49.23761520385742
Epoch: 25, Loss: 42.92498207092285
Epoch: 26, Loss: 49.693812942504884
Epoch: 27, Loss: 44.92315158843994
Epoch: 28, Loss: 41.916252326

(Model(
   (l1): Linear(in_features=100, out_features=1, bias=True)
   (sigmoid): Sigmoid()
 ),
 [74.3392988204956,
  83.12007522583008,
  80.28693923950195,
  75.65507049560547,
  76.28227291107177,
  72.65942840576172,
  71.7201473236084,
  68.56741142272949,
  67.76617736816407,
  64.4251594543457,
  69.57161254882813,
  61.711208724975585,
  63.781436157226565,
  62.946955871582034,
  61.13170280456543,
  55.6065975189209,
  55.38992862701416,
  55.69268112182617,
  52.65674819946289,
  57.7535587310791,
  51.32097473144531,
  53.903232383728025,
  47.677008628845215,
  48.27898254394531,
  49.23761520385742,
  42.92498207092285,
  49.693812942504884,
  44.92315158843994,
  41.916252326965335,
  42.45442714691162,
  37.27211303710938,
  39.91522130966187,
  33.2546667098999,
  28.999010467529295,
  27.835384941101076,
  27.748454093933105,
  23.618341636657714,
  25.359708499908447,
  29.539030838012696,
  15.398544216156006,
  6.696302652359009,
  6.30007266998291,
  19.7566022872

In [8]:
# do not remove – nothing to code here
# run this cell before moving on
# ensure get_accuracy from task 1.5 is defined

def get_accuracy(pred: torch.Tensor, label: torch.Tensor):
    y_pred = (pred > 0.5).long()
    label = label.long()

    return (y_pred == label).float().mean()

with torch.no_grad():
    model.eval()
    for i, data in enumerate(test_loader):
        x, y = data
        pred = model(x)
        y = y.view(-1, 1).long()
        print(pred.dtype, y.dtype)
        acc = get_accuracy(pred, y)
        print(f"Accuracy: {acc}")

torch.float32 torch.int64
Accuracy: 0.9587500095367432
